In [1]:
%cd images

/Users/thom/School/DS2.2/images


In [38]:
import glob
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image
from keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential, load_model
from keras.utils import np_utils
from tensorflow.keras import layers
from keras.layers.core import Dense, Activation
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array

ModuleNotFoundError: No module named 'keras'

In [39]:
files = glob.glob("Fire_images/*.*")
ls_fire = []
for filepath in files:
    ls_fire.append(['Fire_images', os.path.basename(filepath), '1'])

In [21]:
files = glob.glob("Normal_Images_1/*.*")
ls_normal = []
for filepath in files:
    ls_normal.append(['Normal_Images_1', os.path.basename(filepath), '0'])

In [22]:
df_fire = pd.DataFrame(ls_fire, columns=['Folder', 'filename', 'label'])

In [23]:
df_normal = pd.DataFrame(ls_normal, columns=['Folder', 'filename', 'label'])

In [26]:
df = pd.concat([df_fire, df_normal]).reset_index().drop('index', axis=1) # combining the dataframes
df

Folder                                           filename label
0        Fire_images                                      dsc_01001.jpg     1
1        Fire_images                    burning-charcoal-briquettes.jpg     1
2        Fire_images                                  Chimney-Fire3.jpg     1
3        Fire_images                                canada-wildfire.jpg     1
4        Fire_images                          Living_Room_Camp_Fire.jpg     1
..               ...                                                ...   ...
258  Normal_Images_1  amazing-modern-living-room-design-with-white-b...     0
259  Normal_Images_1      813e6b10-848b-478e-a471-480f14eeb271.1.10.jpg     0
260  Normal_Images_1            201504-connect-norway-horizontal-1b.jpg     0
261  Normal_Images_1          690x380-Lake-Tahoe-Summer-Emerald-Bay.jpg     0
262  Normal_Images_1  AAEAAQAAAAAAAAJqAAAAJDAwNDUwNDAwLWUwYTAtNDlmNy...     0

[263 rows x 3 columns]

In [27]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

In [42]:
img = Image.open(os.path.join(df['Folder'][260], df['filename'][260]))
print(img.size)
image_red = img.resize((1024, 1024))
image = img_to_array(image_red)

(930, 581)


NameError: name 'img_to_array' is not defined

In [41]:
print("size: " + str(image_red.size))

size: (1024, 1024)


In [40]:
image.shape

NameError: name 'image' is not defined

In [31]:
def data_gen(df, batch_size):
    
    while True:
        x_batch = np.zeros((batch_size, 1024, 1024, 3))
        y_batch = np.zeros((batch_size, 1))

        for chunk_index in range(len(df)//batch_size):
            start_chunk_index = chunk_index*batch_size
            end_chunk_index = (chunk_index+1)*batch_size

            i = 0
            
            for folder, filename, label in zip(df['Folder'].values[start_chunk_index:end_chunk_index], df['filename'].values[start_chunk_index:end_chunk_index], df['label'].values[start_chunk_index:end_chunk_index]):
                img = Image.open(os.path.join(folder, filename))
                image_red = img.resize((1024, 1024))
                try:
                    x_batch[i] = img_to_array(image_red)
                except:
                    print(f'this file is currupted: {os.path.join(folder, filename)}')
                    os.remove(os.path.join(folder, filename))

                y_batch[i] = label
                i += 1

            yield x_batch, np_utils.to_categorical(y_batch, 2)

In [32]:
df['filename']

0                                          dsc_01001.jpg
1                        burning-charcoal-briquettes.jpg
2                                      Chimney-Fire3.jpg
3                                    canada-wildfire.jpg
4                              Living_Room_Camp_Fire.jpg
                             ...                        
258    amazing-modern-living-room-design-with-white-b...
259        813e6b10-848b-478e-a471-480f14eeb271.1.10.jpg
260              201504-connect-norway-horizontal-1b.jpg
261            690x380-Lake-Tahoe-Summer-Emerald-Bay.jpg
262    AAEAAQAAAAAAAAJqAAAAJDAwNDUwNDAwLWUwYTAtNDlmNy...
Name: filename, Length: 263, dtype: object

In [33]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1024, 1024, 3)))
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu'))  
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.01),
             metrics = ['accuracy'])

NameError: name 'layers' is not defined

In [34]:
batch_size = 50
model.fit(data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=2, verbose=1)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [35]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [36]:
model_2 = tf.keras.Sequential()
model_2.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(1024, 1024, 3)))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Flatten())
model_2.add(layers.Dense(4, activation='relu'))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(2, activation='sigmoid'))

model_2.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.01),
             metrics = ['accuracy'])

NameError: name 'layers' is not defined

In [18]:
model_2.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [19]:
batch_size = 32
model_2.fit(data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=2, verbose=1)

NameError: name 'df_train' is not defined